In [43]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd
from pyproj import CRS, Transformer
import pandas as pd

In [44]:
#Opening and Concatenating all files

u_wind1 = xr.open_dataset('uas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201910-201910.nc.nc4')

u_wind2 = xr.open_dataset('uas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201911-201911.nc.nc4')

u_wind  =  xr.concat([u_wind1, u_wind2], dim='time')

#u_wind = u_wind.sel(time=u_wind['time'][::4])

v_wind1 = xr.open_dataset('vas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201910-201910.nc.nc4')

v_wind2 = xr.open_dataset('vas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201911-201911.nc.nc4')

v_wind  =  xr.concat([v_wind1, v_wind2], dim='time')

#v_wind = v_wind.sel(time=v_wind['time'][::4])




rh1 = xr.open_dataset('hurs_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201910-201910.nc.nc4')

rh2 = xr.open_dataset('hurs_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201911-201911.nc.nc4')

rh  =  xr.concat([rh1, rh2], dim='time')

#rh = rh.sel(time=rh['time'][::4])

sm1 = xr.open_dataset('mrsos_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201910-201910.nc.nc4')

sm2 = xr.open_dataset('mrsos_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201911-201911.nc.nc4')

sm  =  xr.concat([sm1, sm2], dim='time')

#sm = sm.sel(time=sm['time'][::4])


st1 = xr.open_dataset('tas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201910-201910.nc.nc4')

st2 = xr.open_dataset('tas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201911-201911.nc.nc4')

st  =  xr.concat([st1, st2], dim='time')

#st = st.sel(time=st['time'][::4])


C:\Users\timsc\anaconda3\lib\site-packages\xarray\core\concat.py:527: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
C:\Users\timsc\anaconda3\lib\site-packages\xarray\core\concat.py:527: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
C:\Users\timsc\anaconda3\lib\site-packages\xarray\core\concat.py:527: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
C:\Users\timsc\anaconda3\lib\site-packages\xarray\core\concat.py:527: FutureWarning: unique with argument that is not not a Series, Index, 

In [45]:
#Creating Wind Speed and Direction

u = u_wind['uas']
v = v_wind['vas']

# Calculate wind speed
wind_speed = np.sqrt(u**2 + v**2)

# Calculate wind direction in radians
wind_direction_rad = np.arctan2(u, v)

# Convert wind direction to degrees
wind_direction_deg = np.degrees(wind_direction_rad)

# Normalize direction to [0, 360] degrees
wind_direction_deg = (wind_direction_deg + 360) % 360

In [46]:
# Load the shapefile
gdf = gpd.read_file('Grid_0_150.shp')



In [47]:
ft_layer = gpd.read_file('Fuel Layer.shp')

In [48]:
# Define the coordinate reference systems
crs_epsg_3857 = CRS("EPSG:3857")
crs_epsg_4326 = CRS("EPSG:4326")

# Create a transformer object for EPSG:3857 to EPSG:4326
transformer = Transformer.from_crs(crs_epsg_3857, crs_epsg_4326, always_xy=True)

In [49]:
# Extract coordinates
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y

gdf['lon'], gdf['lat'] = transformer.transform(gdf['x'].values, gdf['y'].values)

coords = gdf[['lon', 'lat']]
time_index = u_wind['time'].to_index()

start_date = pd.to_datetime(time_index[0])
end_date = start_date + pd.Timedelta(days=18)
limited_time_index = time_index[(time_index >= start_date) & (time_index < end_date)]

In [50]:
limited_time_index

DatetimeIndex(['2019-10-26 00:00:00', '2019-10-26 01:00:00',
               '2019-10-26 02:00:00', '2019-10-26 03:00:00',
               '2019-10-26 04:00:00', '2019-10-26 05:00:00',
               '2019-10-26 06:00:00', '2019-10-26 07:00:00',
               '2019-10-26 08:00:00', '2019-10-26 09:00:00',
               ...
               '2019-11-09 14:00:00', '2019-11-09 15:00:00',
               '2019-11-09 16:00:00', '2019-11-09 17:00:00',
               '2019-11-09 18:00:00', '2019-11-09 19:00:00',
               '2019-11-09 20:00:00', '2019-11-09 21:00:00',
               '2019-11-09 22:00:00', '2019-11-09 23:00:00'],
              dtype='datetime64[ns]', name='time', length=360, freq=None)

In [51]:
# Create the DataArray and Dataset with the limited time index
data = xr.DataArray(
    np.zeros((len(limited_time_index), len(gdf))),
    dims=['time', 'points'],
    coords={'time': limited_time_index, 'points': np.arange(len(gdf))}
)

ds = xr.Dataset(
    {
        'data': data
    },
    coords={
        'time': limited_time_index,
        'points': np.arange(len(gdf)),
        'x': ('points', gdf['lon']),
        'y': ('points', gdf['lat']), 
        'Is Fire': ('points', gdf['Is Fire']),
        'Aspect': ('points', gdf['Aspect1']),
        'Slope': ('points', gdf['Slope1']),
        'Fuel Type': ('points', ft_layer['Fuel Type1']),
        'TSF': ('points', gdf['Label'])
    }
)

subset_ds = ds

# Convert to DataFrame and reset index
df = subset_ds.to_dataframe().reset_index()

# Format the 'time' column
df['time'] = df['time'].dt.strftime('%Y-%m-%dT%H:%M:%S.000000000')




In [52]:
df['Is Fire'] = df['Is Fire'].fillna(0)


In [53]:
df['TSF'] = df['TSF'].apply(lambda x: '>10' if x == '2015-2016 Prescribed Burn' else '<10')


In [54]:
# Function to get the wind speed for a given time and location
def get_weather_at_location_and_time(var, time, lon, lat):
    # Select the wind speed value for the given time, longitude, and latitude
    #print("time", time)
    return var.sel(time=time, lon=lon, lat=lat, method='nearest').item()



df['wspd'] = df.apply(lambda row: get_weather_at_location_and_time(wind_speed, row['time'], row['x'], row['y']), axis=1)
print("done")
df['wind_direction_deg'] = df.apply(lambda row: get_weather_at_location_and_time(wind_direction_deg, row['time'], row['x'], row['y']), axis=1)
print("done")
df['rh'] = df.apply(lambda row: get_weather_at_location_and_time(rh['hurs'], row['time'], row['x'], row['y']), axis=1)
print("done")

df['sm'] = df.apply(lambda row: get_weather_at_location_and_time(sm['mrsos'], row['time'], row['x'], row['y']), axis=1)
print("done")

df['st'] = df.apply(lambda row: get_weather_at_location_and_time(st['tas'], row['time'], row['x'], row['y']), axis=1)
print("done")


done
done
done
done
done


In [55]:
gdf1 = gpd.read_file('Grid_1_150.shp')
gdf2 = gpd.read_file('Grid_2 150.shp')
gdf3 = gpd.read_file('Grid_3_150.shp')
gdf4 = gpd.read_file('Grid_4_150.shp')
gdf5 = gpd.read_file('Grid_5_150.shp')
gdf6 = gpd.read_file('Grid_6_150.shp')
gdf7 = gpd.read_file('Grid_7_150.shp')
gdf8 = gpd.read_file('Grid_8_150.shp')
gdf9 = gpd.read_file('Grid_9_150.shp')



In [56]:
#Timestep 0

target_time = pd.to_datetime('2019-10-26 05:51:24')  # Replace with your target time

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

closest_timestep


Timestamp('2019-10-26 06:00:00')

In [57]:
threshold_time = closest_timestep

# Filter rows where 'time' is greater than or equal to threshold_time
df = df[df['time'] >= threshold_time]

In [58]:
#Timestep 1

target_time = pd.to_datetime('2019-10-28 13:00:00')  # Replace with your target time

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf1['Is Fire'].values


print(closest_timestep)

2019-10-28 13:00:00


In [59]:
#Timestep 2

target_time = pd.to_datetime('2019-11-01 13:00:00')  # Replace with your target time

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf2['Is Fire'].values


print(closest_timestep)

2019-11-01 13:00:00


In [60]:
#Timestep 3

target_time = pd.to_datetime('2019-11-02 07:00:00')  # Replace with your target time

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf3['Is Fire'].values


print(closest_timestep)

2019-11-02 07:00:00


In [61]:
#Timestep 4

target_time = pd.to_datetime('2019-11-04 04:00:00')  # Replace with your target time

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf4['Is Fire'].values


print(closest_timestep)

2019-11-04 04:00:00


In [62]:
#Timestep 5

target_time = pd.to_datetime('2019-11-06 20:00:00')  # Replace with your target time

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf5['Is Fire'].values


print(closest_timestep)

2019-11-06 20:00:00


In [63]:
#Timestep 6

target_time = pd.to_datetime('2019-11-07 03:30:00')  # Replace with your target time

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf6['Is Fire'].values


print(closest_timestep)

2019-11-07 03:00:00


In [64]:
#Timestep 7

target_time = pd.to_datetime('2019-11-07 21:00:00')  # Replace with your target time

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf7['Is Fire'].values


print(closest_timestep)

2019-11-07 21:00:00


In [65]:
#Timestep 8

target_time = pd.to_datetime('2019-11-08 02:00:00')  # Replace with your target time

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf8['Is Fire'].values


print(closest_timestep)

2019-11-08 02:00:00


In [66]:
#Timestep 9

target_time = pd.to_datetime('2019-11-08 15:00:00')  # Replace with your target time

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf9['Is Fire'].values


print(closest_timestep)

2019-11-08 15:00:00


In [67]:
'''
target_timesteps = ('2019-10-26 06:00:00','2019-10-28 13:00:00', '2019-11-01 13:00:00','2019-11-02 07:00:00',
                   '2019-11-04 04:00:00','2019-11-06 20:00:00', '2019-11-07 03:30:00', 
                   '2019-11-07 21:00:00','2019-11-08 02:00:00', '2019-11-08 15:00:00')
                   
'''

"\ntarget_timesteps = ('2019-10-26 06:00:00','2019-10-28 13:00:00', '2019-11-01 13:00:00','2019-11-02 07:00:00',\n                   '2019-11-04 04:00:00','2019-11-06 20:00:00', '2019-11-07 03:30:00', \n                   '2019-11-07 21:00:00','2019-11-08 02:00:00', '2019-11-08 15:00:00')\n                   \n"

In [68]:
#df = df[df['time'].isin(target_timesteps)]


In [69]:
df['time'].unique()

<DatetimeArray>
['2019-10-26 06:00:00', '2019-10-26 07:00:00', '2019-10-26 08:00:00',
 '2019-10-26 09:00:00', '2019-10-26 10:00:00', '2019-10-26 11:00:00',
 '2019-10-26 12:00:00', '2019-10-26 13:00:00', '2019-10-26 14:00:00',
 '2019-10-26 15:00:00',
 ...
 '2019-11-09 14:00:00', '2019-11-09 15:00:00', '2019-11-09 16:00:00',
 '2019-11-09 17:00:00', '2019-11-09 18:00:00', '2019-11-09 19:00:00',
 '2019-11-09 20:00:00', '2019-11-09 21:00:00', '2019-11-09 22:00:00',
 '2019-11-09 23:00:00']
Length: 354, dtype: datetime64[ns]

In [70]:
# Step 1: Convert 'time' column to datetime format if it's not already
df['time'] = pd.to_datetime(df['time'])

# Step 2: Convert 'time' column to desired string format
df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [71]:
df

,time,points,data,x,y,Is Fire,Aspect,Slope,Fuel Type,TSF,wspd,wind_direction_deg,rh,sm,st
12246,2019-10-26 06:00:00,0,0.0,152.757759,-31.222510,0,294.863708,89.997185,150,<10,3.997191,115.964069,19.588623,18.511719,305.078125
12247,2019-10-26 06:00:00,1,0.0,152.757759,-31.223663,0,306.634094,89.997383,150,<10,3.997191,115.964069,19.588623,18.511719,305.078125
12248,2019-10-26 06:00:00,2,0.0,152.757759,-31.224815,0,257.905243,89.995552,150,<10,3.997191,115.964069,19.588623,18.511719,305.078125
12249,2019-10-26 06:00:00,3,0.0,152.757759,-31.225967,0,293.198608,89.991638,150,<10,3.997191,115.964069,19.588623,18.511719,305.078125
12250,2019-10-26 06:00:00,4,0.0,152.757759,-31.227120,0,254.054611,89.991257,150,<10,3.997191,115.964069,19.588623,18.511719,305.078125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734755,2019-11-09 23:00:00,2036,0.0,152.822438,-31.260529,0,45.000000,89.996536,150,<10,4.239993,57.949854,27.415955,26.722656,294.281250
734756,2019-11-09 23:00:00,2037,0.0,152.822438,-31.261681,0,45.000000,89.997093,150,<10,4.239993,57.949854,27.415955,26.722656,294.281250
734757,2019-11-09 23:00:00,2038,0.0,152.822438,-31.262833,0,92.642548,89.998047,150,<10,4.239993,57.949854,27.415955,26.722656,294.281250
734758,2019-11-09 23:00:00,2039,0.0,152.822438,-31.263984,0,108.434952,89.998512,150,<10,4.239993,57.949854,27.415955,26.722656,294.281250


In [72]:
df.to_csv("Port_Fire_full.csv", index=False)  # Set index=False to exclude row numbers


In [73]:
print("all good!")

all good!
